# Enkratna podloga in napadi nanjo

Cilji laboratorijske vaje so sledeči:
- spoznati postrojitev podatkov za potrebe šifriranja,
- spoznati delovanje enkratne podloge (OTP) in posledično tokovnih šifer,
- spoznati tehniko napada na enkratno podlogo, ko se ključ uporabi večkrat,
- spoznati tehniko napada na enkratno podlogo, ko lahko napadalec spremeni tajnopis.

**POMEMBNO**

Namen te laboratorijske vaje se je seznaniti z načinom ravnanja s šiframi. Naša implementacija šifer bo osnovana na preprostosti in bo narejena **zgolj v pedagoške namene**. Kot taka ni primerna za uporabo v praksi. 

Šifre, ki so za uporabo v praksi primerne in primerno implementirane, bomo spoznali kasneje.

## Ustvarjanje pravih naključnih vrednosti

Naključne vrednosti bomo v Pythonu ustvarili s pomočjo modula `secrets`. Za zdaj bo to najboljši približek pravim naključnim vrednostim; [povezava na dokumentacijo.](https://docs.python.org/3/library/secrets.html)

Najprej modul uvozimo, nato pa imlementiramo funkcijo, katera ustvari seznam naključnih bajtov za podano dolžino.

In [1]:
import secrets

In [2]:
def gen_key(length):
    """Vrne nakljucni niz bajtov podane dolzine"""
    return secrets.token_bytes(length)

## Naloga 1: Postroj podatkov

Vse moderne šifre delujejo na bitih (in bajtih). To pomeni, da moramo visokonivojske podatke (števila, nize, podatkovne strukture) pretvoriti v zaporedje ničel in enic, da jih lahko šifriramo.

Pri pretvorbi nizov uporabimo kodirno shemo. Najbolj pogosti možnosti sta ASCII ter UTF-8. 

Če uporabimo zgolj znake s kodirne tabele ASCII, dobimo pri obeh kodirnih shemah isti rezultat

In [26]:
sporocilo1 = "dober dan"
sporocilo1_ascii = sporocilo1.encode("ascii")
sporocilo1_utf8 = sporocilo1.encode("utf8")

assert sporocilo1_ascii == sporocilo1_utf8

print(sporocilo1)
print(sporocilo1_ascii)
print(sporocilo1_utf8)
print(list(sporocilo1_ascii))

dober dan
b'dober dan'
b'dober dan'
[100, 111, 98, 101, 114, 32, 100, 97, 110]


Če pa uporabimo tudi znake, ki jih na kodirni tabeli ASCII ni, denimo šumnike, pa pride do razlik.

In [27]:
sporocilo2 = "dober večer"
sporocilo2_utf8 = sporocilo2.encode("utf8")
print(sporocilo2_utf8)

b'dober ve\xc4\x8der'


ASCII kodiranje v tem primeru ne deluje: pri kodiranju znaka `č` bo prišlo do napake.

In [28]:
sporocilo2.encode("ascii")

UnicodeEncodeError: 'ascii' codec can't encode character '\u010d' in position 8: ordinal not in range(128)

Včasih želimo bajte predstaviti v šestnajstiškem zapisu. V ta namen uporabimo funkcijo `hex()`.

In [34]:
print(f"Sporočilo '{sporocilo2}' predstavljeno kot:\nSeznam bajtov: {list(sporocilo2_utf8)}\nŠestnajstiški niz: {sporocilo2_utf8.hex()}")

Sporočilo 'dober večer' predstavljeno kot:
Seznam bajtov: [100, 111, 98, 101, 114, 32, 118, 101, 196, 141, 101, 114]
Šestnajstiški niz: 646f626572207665c48d6572


Tako npr. prva dva znaka (`64`) predstavljata šestnajstiški zapis števila `100` ($100 = 6 \cdot 16^1 + 4 \cdot 16^0$), medtem ko vrednost `100` v ASCII/UTF-8 kodiranju predstavlja znak `d`

Funkcija, ki deluje v obratni smeri tj. ki iz seznama bajtov zgradi niz znakov, se imenuje `bytes.decode()`. 

In [24]:
bajti = b'dober ve\xc4\x8der'
print(bajti.decode("utf8"))

dober večer


Kodiranje celih števil je preprosteje: le uporabimo [funkcijo `int.to_bytes()`](https://docs.python.org/3/library/stdtypes.html#int.to_bytes) Funkciji moramo podati vsaj 2 argumenta: število bajtov, ki se naj uporabijo za kodiranje (tj. dolžina) in pravilo po katerem so bajti urejeni tj. ali gre za pravilo debelega ali tankega konca (`big` ali `little`). Opcijsko lahko nastavimo še, ali gre za predznačeno število.

In [11]:
stevilo = 100
print(stevilo.to_bytes(1, 'big').hex())
print(stevilo.to_bytes(1, 'big'))

64
b'd'


Včasih nas zanima, kateri številčni vrednosti (v kodiranju unikod) pripada posamezen znak. To izvemo s pomočjo funkcije `ord()`.

In [12]:
for ch in "dober večer":
    print(ch, ord(ch))

d 100
o 111
b 98
e 101
r 114
  32
v 118
e 101
č 269
e 101
r 114


Funkcija, ki deluje v obratno smer, tj. iz podane unikod vrednosti vrne pripadajoč znak, se imenuje `chr()`.

In [13]:
for code in [100, 111, 98, 101, 114, 32, 118, 101, 269, 101, 114]:
    print(code, chr(code))

100 d
111 o
98 b
101 e
114 r
32  
118 v
101 e
269 č
101 e
114 r


## Naloga 2: Implementacija ekskluzivne disjunkcije

Implementirajte metodo `xor_bytes(s1, s2)`, ki spremje seznama bajtov, `s1` in `s1`, in vrne seznam, kjer so vrednosti posameznih bajtov izračunane kot rezultat operacije ekskluzivne disjunkcije nad posameznimi elementi seznama `s1` in `s2`.

Operacija ekskluzivne disjukncije v Pythonu je mogoča z uporabo znaka `^`:
```python
>>> 1 ^ 2
3
```

V sledeči celici lahko delovanje vaše funkcije preverite na treh enotskih testih. Če testi ne vrnejo napake, jih je vaša implementacija uspešno prestala.

In [35]:
def xor_bytes(s1, s2):
    """Izvede operacijo XOR med podanima seznamoma bajtov in vrne seznam bajtov"""
    return bytes(a ^ b for a, b in zip(s1, s2))


xor_bytes([1, 16], [2, 1])

b'\x03\x11'

In [36]:
assert xor_bytes([1, 2, 3], [3, 2, 1]) == bytes([2, 0, 2])
assert xor_bytes([0, 0, 0], [3, 2, 1]) == bytes([3, 2, 1])
assert xor_bytes([1, 1, 1], [1, 1, 1]) == bytes([0, 0, 0])

## Naloga 3: Šifra enkratne podloge (OTP)

Implementirajte šifrirni in dešifrirni algorite enkratne podloge. Če uporabite rešitev iz prve naloge, bo implementacija trivialno kratka.

In [37]:
def enc_otp(key, pt):
    """Sifrirni algoritem enkratne podloge"""
    return xor_bytes(key, pt)

In [38]:
assert enc_otp([1, 2, 3], [1, 1, 1]) == xor_bytes([1, 2, 3], [1, 1, 1])

In [39]:
def dec_otp(key, ct):
    """Desifrirni algoritem enkratne podloge"""
    return xor_bytes(key, ct)

In [40]:
assert dec_otp([1, 2, 3], [1, 1, 1]) == xor_bytes([1, 2, 3], [1, 1, 1])

## Naloga 4: Šifriranje in dešifriranje

Uporabite metodi šifriranja in dešifriranja iz prejšnje naloge in zašifrirajte sporočilo `Enkratna podloga je popolno tajna šifra.` 

Implementacijo šifriranja podajte v telesu funkcije `example_enc()`. Za ključ uporabite vrednost `3d26ebcbc0b2ad0d15c6be1f6259fd89495451fc2245cd8dad40c480a87bbb3a7525a9ba4abb930417`. Pozor: podano vrednost ključa je potrebno prebrati kot seznam bajtov. To storite s pomočjo funkcije `bytes.fromhex(...)`.

In [20]:
def example_enc():
    msg = "Enkratna podloga je popolno tajna šifra."
    key = bytes.fromhex("3d26ebcbc0b2ad0d15c6be1f6259fd89495451fc2245cd8dad40c480a87bbb3a7525a9ba4abb930417")
    return enc_otp(key, msg.encode("utf8"))

example_enc()

b'xH\x80\xb9\xa1\xc6\xc3l5\xb6\xd1{\x0e6\x9a\xe8i>4\xdcR*\xbd\xe2\xc1.\xab\xa0\xdc\x1a\xd1T\x14\x05l\x1b#\xdd\xe1e9'

In [41]:
assert example_enc().hex() == "784880b9a1c6c36c35b6d17b0e369ae8693e34dc522abde2c12eaba0dc1ad15414056c1b23dde16539"

V telesu funkcije `example_dec()` dešifrirajte sporočilo `784880b9a1c6c36c35b6d17b0e369ae8693e34dc522abde2c12eaba0dc1ad15414056c1b23dde16539` in pri tem uporabite isti ključ kot ste ga pri šifriranju.

In [23]:
def example_dec():
    ct = bytes.fromhex("784880b9a1c6c36c35b6d17b0e369ae8693e34dc522abde2c12eaba0dc1ad15414056c1b23dde16539")
    key = bytes.fromhex("3d26ebcbc0b2ad0d15c6be1f6259fd89495451fc2245cd8dad40c480a87bbb3a7525a9ba4abb930417")
    return dec_otp(key, ct)

example_dec().decode("utf8")

'Enkratna podloga je popolno tajna šifra.'

In [42]:
assert example_dec().decode("utf8") == "Enkratna podloga je popolno tajna šifra."

## Naloga 5: Napad na večkratno podlogo

Poglejmo, kaj lahko naredi napadalec, če isto podlogo uporabimo za šifriranje več sporočil.

### Naloga 5.1: Delno dešifriranje

Za začetek si pripravimo implementacijo dešifriranega algoritma, ki dešifrira tajnopis tudi, če nam kakšen del ključa manjka. Dešifrirani algoritem naj dešifrira kot običajno, edina izjema so znaki, ki je vrednost ključa enaka 0. V tem primeru, naj kot pripadajoč znak v tajnopisu nastavi vrednost `*`.

In [48]:
def dec_otp_partial(key, ct):
    """Desifrira samo tiste znake, kjer kljuc ni 0 -- ce je, kot znak nastavi simbol *"""
    return bytes(k ^ c if k > 0 else ord("*")  for k, c in zip(key, ct))

dec_otp_partial(
    bytes.fromhex("0000ebcbc0b2ad0d15c6be1f6259fd89495451fc0000cd8dad40c480a87bbb3a7525a9ba4abb930000"),
    bytes.fromhex("784880b9a1c6c36c35b6d17b0e369ae8693e34dc522abde2c12eaba0dc1ad15414056c1b23dde16539")
).decode("utf8")

'**kratna podloga je **polno tajna šifr**'

In [49]:
assert dec_otp_partial([1, 0, 1], [2, 2, 2]) == bytes([3, ord("*"), 3])
assert dec_otp_partial([1, 1, 1], [2, 2, 2]) == dec_otp([1, 1, 1], [2, 2, 2])

### Tajnopisi

Tajnopisi so shranjeni v binarni obliki v datotekah `data/ct_i.bin`. Preberimo jih z diska.

In [85]:
def load_cts():
    cipher_texts = []
    for i in range(10):
        with open(f"data/ct_{i}.bin", "rb") as h:
            cipher_texts.append(h.read())
    return cipher_texts

cipher_texts = load_cts()

print("Izpis prvih 50 bajtov tajnopisa šestnajstiško")
for i, c in enumerate(cipher_texts):
    print(f"Tajnopis {i}, dolžina {len(c)}:  {c.hex()[:100]}...")

Izpis prvih 50 bajtov tajnopisa šestnajstiško
Tajnopis 0, dolžina 109:  03aec6ad3319a165f3cc645e758cf996232d4dfa3f0bbf22dc02ddff023fd79d8a36878d18a043e108568a1879ecdda7cffd...
Tajnopis 1, dolžina 109:  03a9c6a13f57bb63fd9a6b552682e0c4363a51f5315db23ec943d8f10235988a8a36898914f041e70a4ddf0672e292f1d1f0...
Tajnopis 2, dolžina 109:  03aec6ad3319a165f3cc675634c9e88b3f2d51f53f59a83bc943deec47219e938a2dcb8c14f05ef70b5cc61e75abdce681ec...
Tajnopis 3, dolžina 109:  00adc0b63319a779f382645e7593eec43f3151e73b45b835860ad1e84b3796c78621cb8518a34ff60319d91873fdd3e9c8bc...
Tajnopis 4, dolžina 109:  00b0dfb23557a26ab28e67573ac9e689297d4be13159ba32ca09cef04d658d86cf3f999d1ca644fd464ddc1b6ee9dda7c7f9...
Tajnopis 5, dolžina 109:  03aec6ad3719bb64fbcc7e49308de18d27341ef53f45ba23c809c2f602369b88993e858418a60ae10919d9113ce5d3a7c8f0...
Tajnopis 6, dolžina 109:  1fa9c6b73f57a46ae68d2e4f3c9ae0876c2e51b13c42b739860dcaee4b369689867b899514aa43fc1552c3546ffbddeac4f2...
Tajnopis 7, dolžina 109:  06e2d9b43b1ba96bfb

Opazimo, da so vsi tajnopisi dolgi 109 bajtov. 

Vaša naloga je, da ugotovite ključ in z njim dešifrirajte sporočila. Pri tem lahko upoštevate še, da so sporočila v Slovenščini ter sestojijo le iz malih črk in presledkov: šumniki, števke in vsa ostala ločila so odstranjeni.

### Naloga 5.2: Funkcija `multiple_xor(pos, cts)`

Implementirajte metodo `multiple_xor(pos, cts)`, ki na vhodu prejme pozicijo znaka v tajnopisu in seznam vseh tajnopisov. Metoda nato vrne bajt ključa oz. vrednost 0, če določi, da bajta v ključu na dani poziciji ni mogoče ugotoviti.

Namig: implementacija je dokaj kratka, če uporabite vgrajeno funkcijo `forall`. V nasprotnem primeru boste potrebovali z vgnezdeno zanko.

In [86]:
def multiple_xor1(pos, cts):
    for c1 in cts:
        if all(c1[pos] ^ c2[pos] >= 65 or c1[pos] ^ c2[pos] == 0 for c2 in cts if c2 != c1):
            return c1[pos] ^ ord(' ')
    
    return 0

def multiple_xor(pos, cts):
    # na dani poziciji znak tajnopisa paroma XORamo z istoleznim znakom v vsakem drugem tajnopisu
    # ce se za nek znak izkaze, da je rezultat te operacije vedno vrednost >= 65 ali == 0
    # potem je znak na dani poziciji presledek
    # znak XOR-amo s presledkom, dobimo kljuc in ga vrnemo
    # v nasprotnem primeru za znak kljuca vrnemo vrednost 0
    for c1 in cts:
        is_c1_space = True
        
        for c2 in cts:
            if c1 == c2:
                continue
            
            if 0 < c1[pos] ^ c2[pos] < 65:
                is_c1_space = False
            
        if is_c1_space:
            return c1[pos] ^ ord(' ')    

    return 0

def guess_key(ciphertexts):
    key = bytearray(len(ciphertexts[0]))

    for i in range(len(ciphertexts[0])):
        key[i] = multiple_xor1(i, ciphertexts)
    return key


gk = guess_key(cipher_texts)
for c in cipher_texts:
    print(dec_otp_partial(gk, c).decode("ascii"))

*l***ni*a j* e*ropska **zava z zemlj*pi*no lego na *krajnem *e*er* s*edozemlj* in na *kr*jnem *ug* sred*je e*
*k*** s*ove*sk* zgodov**o so pomembn* k*lturni vpli*i prihaj*l* i* s*ednjeevr*pskega *n *penin*ke*a kul*urne*
*l***ni*a i*a *ospodar**o ureditev k* t*melji na pr*stem trg* *da* j* po medn*rodnih *er*lih u*rs*ena m*d dr*
*o***no*anj* z* sloven** izvira iz b*se*e slovani i*vor te p* *e *ej*sen je i*peljank* i* kore*a *lov k* je *
*r*** j* bi*o *me upor**ljeno za drz*vn* tvorbo fed*ralna sl*v*ni*a *i je po *klepu z*se*anja *lo*enske*a na*
*l***ns*i p*ed*iki dan**njih slovenc*v *o se na ili*sko ozem*j* s*ov*nije nas*lili v *es*em st*le*ju v *edme*
*k*** l*ta *is*c so bi** napisani br*zi*ski spomeni*i prvi p*s*i *ok*ment v s*ovensci*i *n prv* s*ovans*i za*
* ***la*i n*ro*ov so m** stevilnimi *ar*di tudi slo*enci s p*l*ti*ni* program*m zahte*al* zedi*je*o slo*enij*
*e***af*ko *ez* sloven**a v srednji *vr*pi oziroma * jugovzh*d*i *vr*pi na st*ciscu a*p *inars*eg* gors*va p*
*a***c *lo

Nekatere bajte v ključih ugotovi algoritem, ostale pa poskusite dopolniti sami. 

## Naloga 6: Gnetljivost tajnopisa

Pri zadnji nalogi bomo kot napadalec spremenili tajnopis, tako da bo sprememba vidna v čistopisu. Zgodba je sledeča.

Ana želi poslati zaupno pošto Boru, vas, ki igrate vlogo napadalca Nandija, pa vsebina sporočila _zares_ zanima.

Na srečo vam gre nekaj reči na roko. 

Prvič, Anin računalnik nima internetne povezave, vaš mobilni telefon pa. Zato ji prijazno ponudite, da zanjo postavite mobilno dostopno točko, preko katere se bo lahko povezala v internet in dostopala do poštnega strežnika. Poštni strežnik bo nato sporočilo dostavil Boru. 

Ana in Bor uporabljata poseben protokol za pošto: protokol FMTP -- Funny Mail Transfer Protocol.  Gre za preprost besedilno-osnovan protokol: prva vrstica označuje naslov prejemnika, druga naslov pošiljatelja, tretja zadevo, četrta je vedno prazna in na koncu je sporočilo.

Vse, kar mora Anin poštni odjemalec storiti, da pošlje sporočilo, je, da strežniku FMTP dostavi besedilni niz, podoben naslednjemu.

```txt
prejemnik@enadomena.com
posiljatelj@drugadomena.com
Zadeva sporocila

<Samo sporocilo>
```

Vse morebitne predhodne ali zaključne presledke v vsaki vrstici poštni strežnik ignorira oz. odstrani pred obdelavo. Na primer, e-pošto zgoraj bi lahko napisali kot spodnjo in ne bi bilo nobene razlike.


```txt
                    prejemnik@enadomena.com               
    posiljatelj@drugadomena.com                    
Zadeva sporocila

<Samo sporocilo>
```

Tretjič, Ana vam v dobri veri -- kakšna naivnost! -- pove, da pošilja elektronsko sporočilo Boru in da uporablja protokol FMTP. (Torej poznate strukturo sporočila in vsebino prve vrstice čistopisa.)

Četrtič, Ana uporablja enkratno podlogo, mehanizmov za zagotavljanje celovitosti pa ni.

Ker nastopate v vlogi **posrednika**, lahko sedaj spremenite sporočilo tako, da strežnik FMTP ne bo poslal pošte Boru, temveč jo bo poslal na naslov, ki ga nadzirate vi, npr. `nandi@obvlada.si`.

In [71]:
# Ana in Bor si enkrat tedensko v zivo izmenjata 1000 bajtov
# nakljucnih vrednosti za morebitne potrebe sifriranja
ana_bor_psk = gen_key(1000)

In [73]:
# Ana pripravi sporočilo
email = """bor@student.uni-lj.si
ana@student.uni-lj.si
Hej

Na faksu si pozabil kapo.""".strip()

# In ga šifrira z enkratno podlogo
ct = enc_otp(ana_bor_psk, email.encode("utf8"))

# Nato ga pošlje (to simuliramo) na streznik FMTP
print("Ana pošilja šifrirano sporočilo na strežnik")
print("Tajnopis (HEX):", ct.hex())

Ana pošilja šifrirano sporočilo na strežnik
Tajnopis (HEX): 26a1693affc389cdbe48497f04f7cd983b3de45adc7aff0feb72d29ec77cb35d2673b777f51672e41f8b982303403066c19743de8cf46cc4c20c9492317b68d295f031c3ccacdc4d7156


Sedaj je na potezi napadalec Nandi.

Implementirajte funckijo `change_ct(ct, new_email)`, ki na vhodu prejme tajnopis in email naslov. Spremenite tajnopis tako, da ga bo strežnik FMTP še vedno brez dešifriral, a kot prejemnik ne bo naveden Bor, temveč email naslov, ki je podan v argumentu `new_email`. Predpostavite lahko, da bo `new_email` vedno krajši ali enak od naslova `bor@student.uni-lj.si`.

In [83]:
def change_ct(ct, new_email):
    cur = "bor@student.uni-lj.si"
    new_email = new_email.ljust(len(cur))
    
    l = bytearray(ct)
    
    for i, (c, n) in enumerate(zip(cur, new_email)):
        l[i] = l[i] ^ ord(c) ^ ord(n)
        
    return l

Z poganjanjem spodnje celice lahko preverite, ali vaša funkcija deluje pravilno. 

In [82]:
def fmtp_receive(key, ct):
    print("Streznik FMTP prejel sporocilo:", ct.hex())
    pt = dec_otp(key, ct)
    print("Desifrirano sporocilo:")
    print(pt.decode("utf8"))

fmtp_receive(ana_bor_psk, change_ct(ct, "nandi@obvlada.si"))

Streznik FMTP prejel sporocilo: 2aaf751ee5f793cbad4a5c3510b7d7dc7777ea09957aff0feb72d29ec77cb35d2673b777f51672e41f8b982303403066c19743de8cf46cc4c20c9492317b68d295f031c3ccacdc4d7156
Desifrirano sporocilo:
nandi@obvlada.si     
ana@student.uni-lj.si
Hej

Na faksu si pozabil kapo.
